# Vision Model Selection

Goals:
 - Speed
 - Accuracy

* Ideally this model can be deployed on a future website that requires no GPU to run and has as close to a real time response as possible

Model Candidates:
 - EfficientNet V2 variants https://docs.pytorch.org/vision/stable/models/efficientnetv2.html
 - MobileNetV3  https://docs.pytorch.org/vision/stable/models/mobilenetv3.html
 - ResNet https://docs.pytorch.org/vision/stable/models/resnet.html
 - Vit https://docs.pytorch.org/vision/stable/models/vision_transformer.html

* May expand to more nuance models from hugging face, but I want to start with these as a baseline

In [ ]:
# get personal MLOps Utils
from pathlib import Path
import requests
import os
import zipfile

utils_path = Path("utils/")
utils_path.mkdir(exist_ok=True, parents=True)

with open(utils_path / "utils.zip", "wb") as f:
  request = requests.get("https://github.com/BrandonFors/MLOps_Utils/raw/main/utils.zip")
  print("Downloading utils")
  f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(utils_path / "utils.zip", "r") as zip_ref:
  print("Unzipping utils")
  zip_ref.extractall(utils_path)

# Remove .zip file
os.remove(utils_path / "utils.zip")

Unzipping utils


In [ ]:
import torch
import torchvision

In [ ]:
try:
    from torchinfo import summary
except:
    !pip install  torchinfo
    from torchinfo import summary

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# torch.set_default_device(device=device)

## Prep datasets and transforms

In [ ]:
from datasets import load_dataset, ClassLabel

train_dataset = load_dataset("BrandonFors/Plant-Diseases-PlantVillage-Dataset", split="train")
# train_dataset.set_format(type="torch", columns = ["image", "label"])

test_dataset = load_dataset("BrandonFors/Plant-Diseases-PlantVillage-Dataset", split="test")
# test_dataset.set_format(type="torch", columns = ["image", "label"])

# Get the class names from the dataset and create int labels for the classes
class_names = train_dataset.features["label"].names
class_label = ClassLabel(names=class_names)

# apply the int class labels to the dataset
train_dataset = train_dataset.cast_column("label", class_label)
test_dataset = test_dataset.cast_column("label", class_label)

train_dataset, test_dataset, class_names



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/170M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43456 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10849 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/43456 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/10849 [00:00<?, ? examples/s]

(Dataset({
     features: ['image', 'label'],
     num_rows: 43456
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 10849
 }),
 ['Apple___Apple_scab',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  'Apple___healthy',
  'Blueberry___healthy',
  'Cherry_(including_sour)___Powdery_mildew',
  'Cherry_(including_sour)___healthy',
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
  'Corn_(maize)___Common_rust_',
  'Corn_(maize)___Northern_Leaf_Blight',
  'Corn_(maize)___healthy',
  'Grape___Black_rot',
  'Grape___Esca_(Black_Measles)',
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
  'Grape___healthy',
  'Orange___Haunglongbing_(Citrus_greening)',
  'Peach___Bacterial_spot',
  'Peach___healthy',
  'Pepper,_bell___Bacterial_spot',
  'Pepper,_bell___healthy',
  'Potato___Early_blight',
  'Potato___Late_blight',
  'Potato___healthy',
  'Raspberry___healthy',
  'Soybean___healthy',
  'Squash___Powdery_mildew',
  'Strawberry___Leaf_scorch',
  'Strawberry___healthy'

In [ ]:
# Get model weights from torchvision

### EffNetV2 - S
effnetv2_s_weights = torchvision.models.EfficientNet_V2_S_Weights.DEFAULT
effnetv2_s_auto_transforms = effnetv2_s_weights.transforms()

### MobileNetV3 - S
mobilenetv3_s_weights = torchvision.models.MobileNet_V3_Small_Weights.DEFAULT
mobilenetv3_s_auto_transforms = mobilenetv3_s_weights.transforms()

### ResNet18
resnet18_weights = torchvision.models.ResNet18_Weights.DEFAULT
resnet18_auto_transforms = resnet18_weights.transforms()

### ViT base 16
vitb16_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
vitb16_auto_transforms = vitb16_weights.transforms()

In [ ]:
# Compose  train transforms that incorperate trivial augment
from torchvision import transforms

### EffnetV2
effnetv2_s_train_transforms = transforms.Compose([
  transforms.TrivialAugmentWide(),
  effnetv2_s_auto_transforms
])

### MobileNetV3 - S
mobilenetv3_s_train_transforms = transforms.Compose([
  transforms.TrivialAugmentWide(),
  mobilenetv3_s_auto_transforms
])

### ResNet18
resnet18_train_transforms = transforms.Compose([
  transforms.TrivialAugmentWide(),
  resnet18_auto_transforms
])

### ViT base 16
vitb16_train_transforms = transforms.Compose([
  transforms.TrivialAugmentWide(),
  vitb16_auto_transforms
])

In [ ]:
# test the transforms out
from utils.vision_utils import apply_transforms
test1 = apply_transforms(train_dataset[0], effnetv2_s_train_transforms)["image"].shape
test2 = apply_transforms(train_dataset[0], mobilenetv3_s_train_transforms)["image"].shape
test3 = apply_transforms(train_dataset[0], resnet18_train_transforms)["image"].shape
test4 = apply_transforms(train_dataset[0], vitb16_train_transforms)["image"].shape
test1, test2, test3, test4

(torch.Size([3, 384, 384]),
 torch.Size([3, 224, 224]),
 torch.Size([3, 224, 224]),
 torch.Size([3, 224, 224]))

In [ ]:
# Create the train and test datasets for each model
# apply the train transform to train datasets and the base auto transform to test datasets

### EffnetV2
effnetv2_s_train_dataset = train_dataset.with_transform(lambda x: apply_transforms(x,effnetv2_s_train_transforms))
effnetv2_s_test_dataset = test_dataset.with_transform(lambda x: apply_transforms(x, effnetv2_s_auto_transforms))
### MobileNetV3 - S
mobilenetv3_s_train_dataset = train_dataset.with_transform(lambda x: apply_transforms(x,mobilenetv3_s_train_transforms))
mobilenetv3_s_test_dataset = test_dataset.with_transform(lambda x: apply_transforms(x, mobilenetv3_s_auto_transforms))

### ResNet18
resnet18_train_dataset = train_dataset.with_transform(lambda x: apply_transforms(x,resnet18_train_transforms))
resnet18_test_dataset = test_dataset.with_transform(lambda x: apply_transforms(x, resnet18_auto_transforms))


### ViT base 16
vitb16_train_dataset = train_dataset.with_transform(lambda x: apply_transforms(x,vitb16_train_transforms))
vitb16_test_dataset = test_dataset.with_transform(lambda x: apply_transforms(x, vitb16_auto_transforms))

## Prep Models

In [ ]:
# get all of the models
effnetv2_s = torchvision.models.efficientnet_v2_s(weights=effnetv2_s_weights)
effnetv2_s.name = "effnetv2_s_plant_disease"
mobilenetv3_s = torchvision.models.mobilenet_v3_small(weights=mobilenetv3_s_weights)
mobilenetv3_s.name = "mobilenetv3_s_plant_disease"
resnet18 = torchvision.models.resnet18(weights = resnet18_weights)
resnet18.name = "resnet18_plant_disease"
vitb16 = torchvision.models.vit_b_16(weights = vitb16_weights)
vitb16.name = "vitb16_plant_disease"

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


100%|██████████| 82.7M/82.7M [00:01<00:00, 73.7MB/s]


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 54.5MB/s]


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 79.8MB/s]


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:03<00:00, 88.5MB/s]


In [ ]:
# Modify the classification heads of each of the models so that they output our classes
from torch import nn
### effnetv2
in_features = effnetv2_s.classifier[-1].in_features
effnetv2_s.classifier[-1] = nn.Linear(in_features=in_features, out_features=len(class_names))

### mobilenetv3_s
in_features = mobilenetv3_s.classifier[-1].in_features
mobilenetv3_s.classifier[-1] = nn.Linear(in_features=in_features, out_features=len(class_names))

### resnet18
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features=in_features, out_features=len(class_names))

### vitb17
in_features = vitb16.heads[-1].in_features
vitb16.heads[-1] = nn.Linear(in_features=in_features, out_features=len(class_names))

In [ ]:
from torchinfo import summary

# Print out a summary of the models (just put one here and modified to look at all the models)
summary(model=mobilenetv3_s,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
MobileNetV3 (MobileNetV3)                                    [32, 3, 224, 224]    [32, 38]             --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 576, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 16, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 16, 112, 112]   432                  True
│    │    └─BatchNorm2d (1)                                  [32, 16, 112, 112]   [32, 16, 112, 112]   32                   True
│    │    └─Hardswish (2)                                    [32, 16, 112, 112]   [32, 16, 112, 112]   --                   --
│    └─InvertedResidual (1)                                  [32, 16, 112, 112]   [32, 16, 56,

## Create functions to help train the model and track metrics

In [ ]:
# Create  a save checkpoint function
from pathlib import Path
def save_checkpoint(model:nn.Module,
                    model_name:str,
                    optimizer:torch.optim.Optimizer,
                    scaler: torch.amp.GradScaler,
                    epoch:int):
  """Saves model checkpoints in a checkpoints folder

  Args:
    model (nn.Module): The model to be checkpointed
    model_name (str): The name of the model to be used in the file path created
    optimizer (torch.optim.Optimizer): The optimizer to be checkpointed
    scaler (torch.amp.GradScaler): The scaler to be checkpointed
    epoch (int): the epoch the model is on when being checkpointed
  Returns:
    None
  """

  checkpoint = {
    "epoch": epoch,
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict":optimizer.state_dict(),
    "scalar_state_dict": scaler.state_dict()
  }
  checkpoint_dir = Path("checkpoint/")
  model_dir = checkpoint_dir / model_name

  model_dir.mkdir(exist_ok=True, parents=True)
  save_path = model_dir / f"{model_name}_checkpoint.pth"
  torch.save(checkpoint, save_path)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a SummaryWriter() that can save summarys to a directory.

    Args:
        experiment_name (str): name of experiment
        model_name (str): name of model
        extra (str = None): anything extra to add to the directory

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter()
    """
    import os

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [ ]:
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    if not (model_name.endswith(".pth") or model_name.endswith(".pt")):
      model_name = f"{model_name}.pth"

    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
from tqdm.auto import tqdm
def train(model:nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          epochs:int,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          scaler: torch.amp.GradScaler,
          device:torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter = None,
          completed_epochs: int = 0):
  """Trains a pytorch model for an epoch
  Args:
    model (nn.Module): the model to be trained
    train_dataloader (DataLoader): the dataloader to be used to train the model
    test_dataloader (DataLoader): the dataloader to be used to test the model
          expects dataloaders to return a dict containing an "image" and "label" column
    loss_fn (nn.Module): the loss function to be used to train the model
    optimizer (Optimizer): the optimizer to be used with the model
    scalar (torch.amp.GradScalar):
    completed_epochs
    device (device): the device that calculations will take place on
    experiment_name (str): a name for your training experiment
    writer (torch.utils.tensorboard.writer.SummaryWriter = None): the SummaryWriter that will be used to save the model metrics
    completed_epochs (int = 0): use if training a model form a checkpoint

  Returns:
    None
  """

  total_steps = 0

  for epoch in tqdm(range(completed_epochs+1, epochs+1)):
    model.to(device)
    print(f"Epoch : {epoch}")
    print("-"*50)

    model.train()
    train_loss = 0
    train_acc = 0

    for batch_step, batch in tqdm(enumerate(train_dataloader, start=1), total=len(train_dataloader)):
      # zero the optimizer
      optimizer.zero_grad()
      # move all the items to be used in computation to the device
      batch["image"], batch["label"] = batch["image"].to(device), batch["label"].to(device)
      # enable FP16 mixed precision
      with torch.amp.autocast(device):
        # forward pass
        logits = model(batch["image"])
        # compute the loss
        loss = loss_fn(logits, batch["label"])
      train_loss += loss.item()
      #scale the loss and pass the loss backward
      scaler.scale(loss).backward()
      # step the optimizer
      scaler.step(optimizer)
      scaler.update()

      total_steps += 1
      # calculate the accuracy of the batch
      pred_class = torch.argmax(torch.softmax(logits, dim=1), dim=1)
      train_acc += (pred_class == batch["label"]).sum().item()/len(logits)

    # Get accuracy and average loss per batch
    train_loss = train_loss / len(train_dataloader)
    train_acc = train_acc / len(train_dataloader)
    print(f"Train Loss: {train_loss: .4f} | Train Accuracy: {train_acc: .4f}")

    test_loss = 0
    test_acc = 0
    model.eval()
    for batch_step, batch in tqdm(enumerate(test_dataloader, start=1), total=len(test_dataloader)):
      # move all the items to be used in computation to the device
      batch["image"], batch["label"] = batch["image"].to(device), batch["label"].to(device)
      # activate inference mode
      with torch.inference_mode():
        # enable FP16 mixed precision
        with torch.amp.autocast(device):
          # forward pass
          logits = model(batch["image"])
          # compute the loss
          loss = loss_fn(logits, batch["label"])
        test_loss += loss.item()
        # calculate the accuracy of the batch
        pred_class = torch.argmax(torch.softmax(logits, dim=1), dim=1)
        test_acc += (pred_class == batch["label"]).sum().item()/len(logits)
    # Get accuracy and average loss per test step
    test_loss = test_loss / len(test_dataloader)
    test_acc = test_acc / len(test_dataloader)
    print(f"Test Loss: {test_loss: .4f} | Test Accuracy: {test_acc: .4f}")
    print("")
    if writer:
      # Add results to SummaryWriter
      writer.add_scalars(main_tag="Loss",
                          tag_scalar_dict={"train_loss": train_loss,
                                          "test_loss": test_loss},
                          global_step=epoch)
      writer.add_scalars(main_tag="Accuracy",
                          tag_scalar_dict={"train_acc": train_acc,
                                          "test_acc": test_acc},
                          global_step=epoch)

      # Close the writer
      writer.close()
    # save the checkpoint for the model
    save_checkpoint(model=model,
                    model_name=model.name,
                    optimizer=optimizer,
                    scaler=scaler,
                    epoch=epoch)
  # save model when training is completed
  save_model(model=model,
             target_dir="models",
             model_name=model.name)


## Experiments

All Experiments will use:
 - CrossEntropyLoss
 - an Adam optimizer with lr = 1e-4 (everything else default)
 - a grad scaler to support mixed precision

### EffNetV2-S

In [ ]:
# Create loss function and optimizer
effnetv2_s_loss_fn = nn.CrossEntropyLoss()
effnetv2_s_optimizer = torch.optim.Adam(effnetv2_s.parameters(), lr = 1e-4)
# compile the model
effnetv2_s.compile()
# create the grad scaler to scale mixed precision loss
effnetv2_s_scaler = torch.amp.GradScaler(device)
# Create a writer to save experiment metrics
effnetv2_s_writer = create_writer(experiment_name="plantvillage",
                                      model_name=effnetv2_s.name)

[INFO] Created SummaryWriter, saving to: runs/plantvillage/effnetv2_s_plant_disease...


In [ ]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

# create dataloaders
effnetv2_s_train_dataloader = DataLoader(dataset=effnetv2_s_train_dataset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=NUM_WORKERS)
effnetv2_s_test_dataloader = DataLoader(dataset=effnetv2_s_test_dataset,
                                           batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS)

In [ ]:
NUM_EPOCHS = 5

# Train the model for 5 epochs
train(model=effnetv2_s,
      train_dataloader=effnetv2_s_train_dataloader,
      test_dataloader=effnetv2_s_test_dataloader,
      epochs=NUM_EPOCHS,
      loss_fn=effnetv2_s_loss_fn,
      optimizer=effnetv2_s_optimizer,
      scaler=effnetv2_s_scaler,
      device=device,
      writer= effnetv2_s_writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch : 1
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

W0823 17:38:06.356000 178 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


Train Loss:  0.2978 | Train Accuracy:  0.9295


...

Test Loss:  0.0115 | Test Accuracy:  0.9964

Epoch : 5
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

Train Loss:  0.0267 | Train Accuracy:  0.9922


  0%|          | 0/340 [00:00<?, ?it/s]

Test Loss:  0.0144 | Test Accuracy:  0.9963

[INFO] Saving model to: models/effnetv2_s_plant_disease.pth


### MobileNetV3-S

In [ ]:
# create model loss and optimizer, then compile
mobilenetv3_s_loss_fn = nn.CrossEntropyLoss()
mobilenetv3_s_optimizer = torch.optim.Adam(mobilenetv3_s.parameters(), lr = 1e-4)
mobilenetv3_s.compile()
#create scalar for mixed precision
mobilenetv3_s_scaler = torch.amp.GradScaler(device)
# Create a writer to save experiment metrics
mobilenetv3_s_writer = create_writer(experiment_name="plantvillage",
                                      model_name=mobilenetv3_s.name)

[INFO] Created SummaryWriter, saving to: runs/plantvillage/mobilenetv3_s_plant_disease...


In [ ]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

mobilenetv3_s_train_dataloader = DataLoader(dataset=mobilenetv3_s_train_dataset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                            num_workers=NUM_WORKERS)
mobilenetv3_s_test_dataloader = DataLoader(dataset=mobilenetv3_s_test_dataset,
                                           batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS)

In [ ]:
NUM_EPOCHS = 5

train(model=mobilenetv3_s,
      train_dataloader=mobilenetv3_s_train_dataloader,
      test_dataloader=mobilenetv3_s_test_dataloader,
      epochs=NUM_EPOCHS,
      loss_fn=mobilenetv3_s_loss_fn,
      optimizer=mobilenetv3_s_optimizer,
      scaler=mobilenetv3_s_scaler,
      device=device,
      writer= mobilenetv3_s_writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch : 1
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

W0824 00:37:18.850000 208 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


Train Loss:  0.5660 | Train Accuracy:  0.8500


...

Test Loss:  0.0225 | Test Accuracy:  0.9927

Epoch : 5
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

Train Loss:  0.0637 | Train Accuracy:  0.9800


  0%|          | 0/340 [00:00<?, ?it/s]

Test Loss:  0.0218 | Test Accuracy:  0.9920

[INFO] Saving model to: models/mobilenetv3_s_plant_disease.pth


### ResNet18

In [ ]:
# create model loss and optimizer, then compile
resnet18_loss_fn = nn.CrossEntropyLoss()
resnet18_optimizer = torch.optim.Adam(resnet18.parameters(), lr = 1e-4)
resnet18.compile()
#create scalar for mixed precision
resnet18_scaler = torch.amp.GradScaler(device)
# Create a writer to save experiment metrics
resnet18_writer = create_writer(experiment_name="plantvillage",
                                model_name=resnet18.name)

[INFO] Created SummaryWriter, saving to: runs/plantvillage/resnet18_plant_disease...


In [ ]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

resnet18_train_dataloader = DataLoader(dataset=resnet18_train_dataset,
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       num_workers=NUM_WORKERS)
resnet18_test_dataloader = DataLoader(dataset=resnet18_test_dataset,
                                      batch_size=BATCH_SIZE,
                                      num_workers=NUM_WORKERS)

In [ ]:
NUM_EPOCHS = 5
train(model=resnet18,
      train_dataloader=resnet18_train_dataloader,
      test_dataloader=resnet18_test_dataloader,
      epochs=NUM_EPOCHS,
      loss_fn=resnet18_loss_fn,
      optimizer=resnet18_optimizer,
      scaler=resnet18_scaler,
      device=device,
      writer= resnet18_writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch : 1
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

W0823 21:21:18.354000 2248 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


Train Loss:  0.3455 | Train Accuracy:  0.9121


...

Test Loss:  0.0286 | Test Accuracy:  0.9917

Epoch : 5
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

Train Loss:  0.0680 | Train Accuracy:  0.9793


  0%|          | 0/340 [00:00<?, ?it/s]

Test Loss:  0.0192 | Test Accuracy:  0.9929

[INFO] Saving model to: models/resnet18_plant_disease.pth


### ViT b 16

In [ ]:
# create model loss and optimizer, then compile
vitb16_loss_fn = nn.CrossEntropyLoss()
vitb16_optimizer = torch.optim.Adam(vitb16.parameters(), lr = 1e-4)
vitb16.compile()
#create scalar for mixed precision
vitb16_scaler = torch.amp.GradScaler(device)
# Create a writer to save experiment metrics
vitb16_writer = create_writer(experiment_name="plantvillage",
                              model_name=vitb16.name)

[INFO] Created SummaryWriter, saving to: runs/plantvillage/vitb16_plant_disease...


In [ ]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

vitb16_train_dataloader = DataLoader(dataset=vitb16_train_dataset,
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       num_workers=NUM_WORKERS)
vitb16_test_dataloader = DataLoader(dataset=vitb16_test_dataset,
                                      batch_size=BATCH_SIZE,
                                      num_workers=NUM_WORKERS)

In [ ]:
NUM_EPOCHS = 5
train(model=vitb16,
      train_dataloader=vitb16_train_dataloader,
      test_dataloader=vitb16_test_dataloader,
      epochs=NUM_EPOCHS,
      loss_fn=vitb16_loss_fn,
      optimizer=vitb16_optimizer,
      scaler=vitb16_scaler,
      device=device,
      writer= vitb16_writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch : 1
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

Train Loss:  0.2647 | Train Accuracy:  0.9272


  0%|          | 0/340 [00:00<?, ?it/s]

...

Test Loss:  0.0611 | Test Accuracy:  0.9791

Epoch : 5
--------------------------------------------------


  0%|          | 0/1358 [00:00<?, ?it/s]

Train Loss:  0.0817 | Train Accuracy:  0.9749


  0%|          | 0/340 [00:00<?, ?it/s]

Test Loss:  0.0210 | Test Accuracy:  0.9934

[INFO] Saving model to: models/vitb16_plant_disease.pth


### Upload results to huggingface

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!git config --global user.name ""
!git config --global user.email ""

In [ ]:
!git clone https://huggingface.co/BrandonFors/effnetv2_s_plant_disease
%cd ./effnetv2_s_plant_disease
!pwd

Cloning into 'effnetv2_s_plant_disease'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (4/4), 1.12 KiB | 1.12 MiB/s, done.
/content/effnetv2_s_plant_disease
/content/effnetv2_s_plant_disease


In [ ]:
!cp /content/models/effnetv2_s_plant_disease.pth .


"*.pth" already supported


In [ ]:
!mkdir runs
!cp -r /content/runs/plantvillage/effnetv2_s_plant_disease ./runs

In [ ]:
# Add and commit
!git add .
!git commit -m "Upload .pth model state dict"
!git push origin main

[main a019359] Upload .pth model state dict
 26 files changed, 78 insertions(+)
 create mode 100644 effnetv2_s_plant_disease.pth
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755971528.f240e06164bc.178.4
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755972093.f240e06164bc.178.9
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755972668.f240e06164bc.178.14
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755973242.f240e06164bc.178.19
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755973825.f240e06164bc.178.24
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_train_acc/events.out.tfevents.1755971528.f240e06164bc.178.3
 create mode 100644 runs/effnetv2_s_plant_disease/Accuracy_train_acc/events.out.tfevents.1755972093.f240e06164bc.178.8
 create mode 100644 runs/effnetv2_s_plan

In [ ]:
%cd /content
!rm -r effnetv2_s_plant_disease/

/content


In [ ]:
!git clone https://huggingface.co/BrandonFors/vitb16_plant_disease
%cd ./vitb16_plant_disease
!pwd

!cp /content/models/vitb16_plant_disease.pth .

!mkdir runs
!cp -r /content/runs/plantvillage/vitb16_plant_disease ./runs

# Add and commit
!git add .
!git commit -m "Upload .pth model state dict"
!git push origin main

Cloning into 'vitb16_plant_disease'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 38 (delta 3), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (38/38), 5.51 KiB | 705.00 KiB/s, done.
/content/vitb16_plant_disease
/content/vitb16_plant_disease
cp: cannot stat '/content/models/vitb16_plant_disease.pth': No such file or directory
mkdir: cannot create directory ‘runs’: File exists
cp: cannot stat '/content/runs/plantvillage/vitb16_plant_disease': No such file or directory
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [ ]:
%cd /content
!rm -r vitb16_plant_disease/

/content


In [ ]:
!git clone https://huggingface.co/BrandonFors/resnet18_plant_disease
%cd ./resnet18_plant_disease
!pwd

!cp /content/models/resnet18_plant_disease.pth .

!mkdir runs
!cp -r /content/runs/plantvillage/resnet18_plant_disease ./runs

# Add and commit
!git add .
!git commit -m "Upload .pth model state dict"
!git push origin main

Cloning into 'resnet18_plant_disease'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 38 (delta 3), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (38/38), 5.50 KiB | 704.00 KiB/s, done.
/content/resnet18_plant_disease
/content/resnet18_plant_disease
cp: cannot stat '/content/models/resnet18_plant_disease.pth': No such file or directory
mkdir: cannot create directory ‘runs’: File exists
cp: cannot stat '/content/runs/plantvillage/resnet18_plant_disease': No such file or directory
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [ ]:
%cd /content
!rm -r resnet18_plant_disease/

/content


In [ ]:
!git clone https://huggingface.co/BrandonFors/mobilenetv3_s_plant_disease
%cd ./mobilenetv3_s_plant_disease
!pwd

!cp /content/models/mobilenetv3_s_plant_disease.pth .

!mkdir runs
!cp -r /content/runs/plantvillage/mobilenetv3_s_plant_disease ./runs

# Add and commit
!git add .
!git commit -m "Upload .pth model state dict"
!git push origin main

Cloning into 'mobilenetv3_s_plant_disease'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 75 (delta 9), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (75/75), 10.87 KiB | 585.00 KiB/s, done.
/content/mobilenetv3_s_plant_disease
/content/mobilenetv3_s_plant_disease
[main 83fe50f] Upload .pth model state dict
 26 files changed, 78 insertions(+)
 create mode 100644 runs/mobilenetv3_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755996187.d162579e09aa.208.5
 create mode 100644 runs/mobilenetv3_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755996364.d162579e09aa.208.10
 create mode 100644 runs/mobilenetv3_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755996543.d162579e09aa.208.15
 create mode 100644 runs/mobilenetv3_s_plant_disease/Accuracy_test_acc/events.out.tfevents.1755996721.d162579e09aa.208.20
 create mode 100644 runs/mobilenetv3_s_plan

In [ ]:
%cd /content
!rm -r ./mobilenetv3_s_plant_disease

/content
